<a href="https://colab.research.google.com/github/atharvadesai1/ChatApp/blob/main/lungs_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/"

'Colab Notebooks'   Lung_Disease_Dataset.zip   lungs_transfer_learning.ipynb


In [ ]:
# import zipfile
# zip_path = "/content/drive/My Drive/Lung_Disease_Dataset.zip"
# extract_path = "/content/Lung_Disease_Dataset"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print("Extraction Complete!")


In [ ]:
# import shutil
# import os

# folder_path = "/content/drive/MyDrive"

# if os.path.exists(folder_path):
#     shutil.rmtree(folder_path)
#     print(f"Deleted: {folder_path}")
# else:
#     print(f"Folder not found: {folder_path}")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_path = '/content/Lung_Disease_Dataset/train'
valid_path = '/content/Lung_Disease_Dataset/val'
test_path = '/content/Lung_Disease_Dataset/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(380, 380),
    batch_size=32,
    class_mode='categorical'
)

validation_set = test_datagen.flow_from_directory(
    valid_path,
    target_size=(380, 380),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(380, 380),
    batch_size=32,
    class_mode='categorical'
)

Found 6054 images belonging to 5 classes.
Found 2016 images belonging to 5 classes.
Found 2025 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *


model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380, 380, 3))

In [ ]:
my_model = Sequential([
    Input(shape=(380, 380, 3)),
    model,
    GlobalMaxPooling2D(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(5, activation='softmax')
])


my_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb4 (Functional)          │ (None, 12, 12, 1792)        │      17,673,823 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling2d                 │ (None, 1792)                │               0 │
│ (GlobalMaxPooling2D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         918,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,596,452 (70.94 MB)

 Trainable params: 18,470,221 (70.46 MB)

 Non-trainable params: 126,231 (493.09 KB)

In [ ]:
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy


metrics = ['accuracy', Precision(), Recall(), AUC()]
#check to see if Adam is overwriting the imported optimizer
if type(Adam) == type(Sequential()):
  print("Adam is overwritten.")
else:
  print("Adam is not overwritten.")

metrics = ['accuracy', Precision(), Recall(), AUC()]

adam_optimizer = Adam(learning_rate=0.001)
my_model.compile(loss=CategoricalCrossentropy(),
                 optimizer=adam_optimizer,
                 metrics=metrics)

Adam is not overwritten.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

In [ ]:
history=my_model.fit(training_set, epochs=10,
             validation_data=validation_set,
             callbacks=[lr_scheduler,es])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


In [ ]:
results = my_model.evaluate(test_set)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")
print(f"Test AUC: {results[4]}")

In [ ]:
import matplotlib.pyplot as plt

def plot_combined_history(history):

    plt.figure(figsize=(12, 6))


    plt.plot(history.history['accuracy'], label='Train Accuracy', linestyle='-', marker='o')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linestyle='--', marker='o')

    # Plot loss
    plt.plot(history.history['loss'], label='Train Loss', linestyle='-', marker='x')
    plt.plot(history.history['val_loss'], label='Validation Loss', linestyle='--', marker='x')

    # Add title and labels
    plt.title('Model Training and Validation Metrics')
    plt.xlabel('Epoch')
    plt.ylabel('Value')

    # Add legend
    plt.legend(loc='best')

    # Display the grid
    plt.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Plot combined metrics
plot_combined_history(history)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np


y_pred = my_model.predict(test_set, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels


y_true = test_set.classes


class_labels = list(test_set.class_indices.keys())


conf_matrix = confusion_matrix(y_true, y_pred_classes)


disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_labels)
disp.plot(cmap='Blues', xticks_rotation='vertical')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
my_model.save('/content/lungs_results.h5')